In [9]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [10]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_sizes, output_size, activation_hidden='relu', activation_output='sigmoid', task='classification', learning_rate=0.01):
        np.random.seed(42)
        self.lr = learning_rate
        self.task = task
        self.activation_hidden = activation_hidden
        self.activation_output = activation_output
        self.weights = []
        self.biases = []
        sizes = [input_size] + hidden_sizes + [output_size]
        for i in range(len(sizes) - 1):
            self.weights.append(np.random.randn(sizes[i], sizes[i+1]) * np.sqrt(2 / sizes[i]))
            self.biases.append(np.zeros((1, sizes[i+1])))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def relu(self, x):
        return np.maximum(0, x)

    def relu_derivative(self, x):
        return (x > 0).astype(float)

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    def forward(self, X):
        self.activations = [X]
        self.z_values = []
        for i in range(len(self.weights)):
            z = np.dot(self.activations[-1], self.weights[i]) + self.biases[i]
            self.z_values.append(z)
            is_output_layer = (i == len(self.weights) - 1)
            if is_output_layer:
                if self.task == 'regression':
                    activation = z
                elif self.activation_output == 'softmax':
                    activation = self.softmax(z)
                elif self.activation_output == 'sigmoid':
                    activation = self.sigmoid(z)
                else:
                    activation = z
            else:
                if self.activation_hidden == 'relu':
                    activation = self.relu(z)
                else:
                    activation = self.sigmoid(z)
            self.activations.append(activation)
        return self.activations[-1]

    def backward(self, X, y, output):
        errors = []
        deltas = []
        m = X.shape[0]

        if self.task == 'classification':
            if self.activation_output == 'softmax':
                delta = (output - y) / m
            else:
                error = y - output
                delta = error * self.sigmoid_derivative(output)
        else:
            error = (output - y) / m
            delta = error

        deltas.append(delta)

        for i in range(len(self.weights) - 2, -1, -1):
            if self.activation_hidden == 'relu':
                deriv = self.relu_derivative(self.activations[i+1])
            else:
                deriv = self.sigmoid_derivative(self.activations[i+1])
            delta = deltas[-1].dot(self.weights[i+1].T) * deriv
            deltas.append(delta)

        deltas.reverse()
        for i in range(len(self.weights)):
            self.weights[i] -= self.lr * self.activations[i].T.dot(deltas[i])
            self.biases[i] -= self.lr * np.sum(deltas[i], axis=0, keepdims=True)

    def train(self, X, y, epochs=2000, batch_size=None):
        for epoch in range(epochs):
            if batch_size is None:
                output = self.forward(X)
                self.backward(X, y, output)
            else:
                permutation = np.random.permutation(X.shape[0])
                X_shuffled = X[permutation]
                y_shuffled = y[permutation]
                for start in range(0, X.shape[0], batch_size):
                    end = start + batch_size
                    xb = X_shuffled[start:end]
                    yb = y_shuffled[start:end]
                    output = self.forward(xb)
                    self.backward(xb, yb, output)

            if epoch % 100 == 0 or epoch == epochs - 1:
                output_full = self.forward(X)
                loss = np.mean(np.square(y - output_full))
                print(f"Epoch {epoch}, Loss: {loss:.4f}")

    def predict(self, X):
        return self.forward(X)


iris = load_iris()
X_iris = iris.data
y_iris = iris.target.reshape(-1, 1)

encoder = OneHotEncoder(sparse_output=False)
y_iris = encoder.fit_transform(y_iris)

X_train_iris, X_test_iris, y_train_iris, y_test_iris = train_test_split(X_iris, y_iris, test_size=0.2, random_state=42)

X_mean = np.mean(X_iris, axis=0)
X_std = np.std(X_iris, axis=0)
X_train_iris = (X_train_iris - X_mean) / X_std
X_test_iris = (X_test_iris - X_mean) / X_std

nn_classification = NeuralNetwork(
    input_size=4,
    hidden_sizes=[10, 10],
    output_size=3,
    activation_hidden='relu',
    activation_output='softmax',
    task='classification',
    learning_rate=0.01
)
nn_classification.train(X_train_iris, y_train_iris, epochs=2000, batch_size=16)

predictions_iris = nn_classification.predict(X_test_iris)
predicted_classes = np.argmax(predictions_iris, axis=1)
true_classes = np.argmax(y_test_iris, axis=1)
accuracy = np.mean(predicted_classes == true_classes)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


data = np.random.rand(1000, 4)
columns = ['Joke1', 'Joke2', 'Joke3', 'Anekdot']
df = pd.DataFrame(data, columns=columns)

X_reg = df[['Joke1', 'Joke2', 'Joke3']].values
y_reg = df['Anekdot'].values.reshape(-1, 1)

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_reg = scaler_X.fit_transform(X_reg)
y_reg = scaler_y.fit_transform(y_reg)

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

скрытые_нейроны = 10
epochs = 1000

nn_regression = NeuralNetwork(
    input_size=3,
    hidden_sizes=[скрытые_нейроны],
    output_size=1,
    activation_hidden='relu',
    activation_output='linear',
    task='regression',
    learning_rate=0.01
)
nn_regression.train(X_train_reg, y_train_reg, epochs=epochs, batch_size=16)

y_pred = nn_regression.predict(X_test_reg)
y_pred = scaler_y.inverse_transform(y_pred)
y_test_orig = scaler_y.inverse_transform(y_test_reg)

print("Real vs Predicted:")
for real, pred in zip(y_test_orig[:100], y_pred[:100]):
    print(f"Real: {real[0]:.2f}, Predicted: {pred[0]:.2f}")



Epoch 0, Loss: 0.2587
Epoch 100, Loss: 0.0423
Epoch 200, Loss: 0.0151
Epoch 300, Loss: 0.0115
Epoch 400, Loss: 0.0102
Epoch 500, Loss: 0.0099
Epoch 600, Loss: 0.0094
Epoch 700, Loss: 0.0092
Epoch 800, Loss: 0.0091
Epoch 900, Loss: 0.0088
Epoch 1000, Loss: 0.0087
Epoch 1100, Loss: 0.0086
Epoch 1200, Loss: 0.0086
Epoch 1300, Loss: 0.0085
Epoch 1400, Loss: 0.0084
Epoch 1500, Loss: 0.0084
Epoch 1600, Loss: 0.0084
Epoch 1700, Loss: 0.0084
Epoch 1800, Loss: 0.0083
Epoch 1900, Loss: 0.0083
Epoch 1999, Loss: 0.0082
Test Accuracy: 100.00%
Epoch 0, Loss: 1.0690
Epoch 100, Loss: 0.9578
Epoch 200, Loss: 0.9524
Epoch 300, Loss: 0.9505
Epoch 400, Loss: 0.9469
Epoch 500, Loss: 0.9458
Epoch 600, Loss: 0.9451
Epoch 700, Loss: 0.9460
Epoch 800, Loss: 0.9446
Epoch 900, Loss: 0.9447
Epoch 999, Loss: 0.9447
Real vs Predicted:
Real: 0.60, Predicted: 0.55
Real: 0.06, Predicted: 0.53
Real: 0.03, Predicted: 0.71
Real: 0.08, Predicted: 0.61
Real: 0.20, Predicted: 0.48
Real: 0.26, Predicted: 0.48
Real: 0.92, Pre